# Session 1

First, make sure that the server and the web interface are running. Then, we instantiate the controller:

In [ ]:
from vivarium.controllers.notebook_controller import NotebookController
import numpy as np
controller = NotebookController()

### Defining agent's behaviors

We can define the behavior of an agent as a Python function taking as argument an `Agent` and returning the values of the left and right motors, in this order. Within the body of the function, one can access all the agents attribute. Usually, defining a behavior requires to access the value of the agent sensors. This done through the `agent.left_prox` and `agent.right_prox` attributes, that return the value of the left and right sensors, respectively.

Let's define four behaviors, here corresponding to the four canonical behaviors of [Braitenberg Vehicles](https://en.wikipedia.org/wiki/Braitenberg_vehicle):

In [ ]:
def aggression(agent):
    return agent.right_prox, agent.left_prox

def fear(agent):
    return agent.left_prox, agent.right_prox

def love(agent):
    return 1. - agent.left_prox, 1. - agent.right_prox

def shy(agent):
    return 1. - agent.right_prox, 1. - agent.left_prox

The code above only declares the behaviors, now we need to attach them to the agents in the simulator. Let's attach the `shy` behavior to the first five agents and set their color to blue ; and the `aggression` behavior to the four last agents and set their color to red:

In [ ]:
for ag in controller.agents[:5]:
    ag.color = 'blue'
    ag.detach_all_behaviors()
    ag.attach_behavior(shy)
for ag in controller.agents[5:]:
    ag.color = 'red'
    ag.detach_all_behaviors()
    ag.attach_behavior(aggression)

All agents are now equipped with a behavior. We can launch the simulation with the code below ; then stop it with the code on the next line.

In [ ]:
controller.run(threaded=True)

In [ ]:
controller.stop()